In [ ]:
f = open("/Users/ans/Documents/kurs_projekat/CollegeMsg.txt","r")
input_nodes_edges = print(f.read())
f.close()

1 2 1082040961
3 4 1082155839
5 2 1082414391
6 7 1082439619
8 7 1082439756
9 10 1082440403
9 11 1082440453
12 13 1082441188
9 14 1082441754
9 15 1082441824
9 16 1082441895
9 17 1082442153
9 14 1082442328
9 18 1082442560
19 18 1082443023
20 21 1082444944
19 22 1082450074
8 23 1082450243
9 24 1082450645
9 22 1082450656
25 21 1082467089
26 7 1082481125
27 28 1082490240
29 25 1082504083
30 31 1082517506
30 31 1082517511
30 31 1082517513
30 31 1082517516
32 33 1082518023
34 35 1082525530
34 33 1082525718
36 37 1082530949
38 39 1082538849
36 40 1082539557
41 15 1082540305
41 11 1082540354
41 14 1082540462
41 13 1082540496
41 39 1082541442
41 42 1082574211
41 43 1082576344
9 40 1082583700
44 45 1082585468
46 22 1082587705
44 46 1082588222
47 46 1082588508
48 46 1082589483
9 49 1082596555
36 50 1082597715
44 51 1082597751
32 52 1082598056
36 32 1082598122
53 54 1082598395
36 55 1082598545
56 57 1082598548
36 32 1082598685
36 58 1082598765
44 59 1082598920
51 58 1082598934
44 14 1082599072
36 5